In [1]:
import exposure_slack_sims as sim
from dataclasses import dataclass
from typing import *
import math

In [2]:
ETHERNET_MTU_STANDARD = 1500
ETHERNET_MTU_JUMBO = 9000

KiB = 2 ** 10
MiB = 2 ** 20
GiB = 2 ** 30

ms = 1/1_000
us = 1/1_000_000

class KeyAmountCalcScheme:
    def n_in_queue_keys(self, queue_len: int, elem_size_b: int) -> int: ...
    def n_buffer_keys(self) -> int: ...

class ExactKeyAmountPerQueue(KeyAmountCalcScheme):
    # Must include at least one 'buffer' key e.g. for a queue
    # AAAAAAABBBBBBBCCCCCCCDD    <- new elements can be created in D  
    # |-----||-----||-----||-----|
    # If you may be consuming OoO, it may be wise to keep multiple buffer keys e.g.
    # AAABBBBBCCCCCCCDDDDDDDE    <- A not consumed yet, B not consumed yet, C and D are full, would need E
    
    def __init__(self, n_in_queue_keys: int, n_buffer_keys: int=1):
        self.in_queue = n_in_queue_keys
        self.buffer = n_buffer_keys

    def n_in_queue_keys(self, queue_len: int, elem_size_b: int) -> int:
        return self.in_queue
    def n_buffer_keys(self) -> int:
        return self.buffer

class KeyAmountTargetingOverheadPerQueue(KeyAmountCalcScheme):
    targeted_overhead_b: int

    def __init__(self, targeted_overhead_b: int, n_buffer_keys: int=1):
        self.targeted_overhead_b = targeted_overhead_b
        self.buffer = n_buffer_keys

    def n_in_queue_keys(self, queue_len: int, elem_size_b: int) -> int:
        # overhead = (queue_len // n_queue_keys) * elem_size_b
        # overhead / elem_size_b = queue_len // n_queue_keys
        # n_queue_keys / queue_len = elem_size_b / overhead
        # n_queue_keys = queue_len * elem_size_b / overhead
        n_keys = queue_len * elem_size_b // self.targeted_overhead_b
        print(f"KeyAmountTargetingOverhead selecting {n_keys} in-queue keys")
        if self.targeted_overhead_b <= elem_size_b:
            print(f"WARNING: Selecting a targeted overhead of {self.targeted_overhead_b} for a simulation with elem size {elem_size_b} results in overprovisioning keys")
        return n_keys
    def n_buffer_keys(self) -> int:
        return self.buffer

@dataclass
class SimConfig:
    queue_len: int
    elem_size_b: int
    n_queues: int
    key_amount: KeyAmountCalcScheme
    
    def run(self, ticks=10_000) -> Tuple['SimConfig', sim.SaturatedQueueStats]:
        n_in_queue_keys = self.key_amount.n_in_queue_keys(self.queue_len, self.elem_size_b)
        n_buffer_keys = self.key_amount.n_buffer_keys()
        stats = sim.SaturatedQueue(
            key_alloc=sim.RoundRobinLimitKeyAllocator(num_keys=(n_in_queue_keys + n_buffer_keys), key_limit=max(1, math.ceil(self.queue_len / n_in_queue_keys))),
            elem_consume=sim.FifoElementConsumer(num_elems=self.queue_len),
        ).run(ticks)
        return (self, stats)

def log_stats(sim_stats: Tuple[SimConfig, sim.SaturatedQueueStats]):
    sim.log_stats(sim_stats[1], elem_size=sim_stats[0].elem_size_b, n_queues=sim_stats[0].n_queues)

def run_and_log(sim: SimConfig):
    sim_stats = sim.run()
    log_stats(sim_stats)

# NVMe

Consider 1us latency. If that's accurate (TODO source): 1us = 1 millionth of a second = 1 per MHz = 1000 cycles on a 1GHz, 5000 cycles on a 5GHz. 
DRAM DDR5 latency is 10-20ns but a) that's latency at the controller I think and b) that's pipelined, it's not like you only have 50-100 DRAM writes between issuing the NVMe request and it coming back.
**Unfortunately this is not accurate - haasWhatModernNVMe2023 measures latency at 60-100us**.

Anecdotal sources cap out at 64 and claim it's hard to hit that randomly anyway:
- https://uk.pcpartpicker.com/benchmarks/methodology/storage/
- https://www.tomshardware.com/features/ssd-benchmarks-hierarchy
    > We've sorted by the random QD1 IOPS results for the tables — the geometric mean of both the read and write IOPS, to be precise. This is one of the more realistic representations of overall SSD performance, even if it's a synthetic test, as it's difficult to game the system. Lots of manufacturers will test random IO performance at queue depths of 32 or even 256 because that makes everything look much faster, but in the real world, random queue depths are mostly at QD1 and almost never go beyond QD4.

These anecdotal setups use 4KiB for random and 1MiB for sequential I/O.

Sequential I/O is more likely to saturate the queue????? so assume 1MiB.

## Dell

<https://infohub.delltechnologies.com/en-us/l/an-in-depth-overview-of-nvme-and-nvme-of/queue-implementations/#:~:text=The%20ability%20of%20NVMe%20to%20create%20many,queue%20depths%20ranging%20from%2032%20to%202%2C048.>

> The ability of NVMe to create many queues with large queue depths ensures that sufficient commands can be queued to use compute resources efficiently and use all available fabric bandwidth fully. While NVMe supports over 65,000 queues with queue depth support over 65,000 commands per queue, the typical NVMe implementation attempts to allocate a maximum of one queue pair per host CPU core. This results in implementations with normal ranges being from four to 256 queues with queue depths ranging from 32 to 2,048.

## zouDirectNVMHardwareacceleratedNVMe2022

> NVMe supports 64K pairs of I/O queues, each with 64K entries, which cannot fit in on-chip memories. However, in most use cases, a very shallow I/O queue is sufficient. Intel conducted a series of SSD benchmarking with different workloads [9] and got a conclusion that the NVMe performance asymptotically saturates as the queue depth increases. Our experiments also verify that the throughput slightly changes when the I/O queue depth grows larger than 64, which will be further explained and validated through approximate mathematical modeling in Section 7.1.

Note that this is inside their own custom system on an FPGA, so it's only good as a ballpark

## haasWhatModernNVMe2023

2023 paper showing having ~3000 requests outstanding on 8 NVMe SSDs saturates them in Figure 4, hence we can likely assume 3000/8 = 375 saturates one 2023 NVMe SSD.


## intelPerformanceBenchmarkingPCIe2015

2015 whitepaper showing NVMes of the time used on the order of 50-100 queue depth

## harrisFreeBSDNVMExpress2018

As of 2018 in vanilla FreeBSD each queue has 128 outstanding items.

# Netflix FreeBSD NVMe Workload

<https://freebsdfoundation.org/wp-content/uploads/2018/08/FreeBSD-and-NVM-Express.pdf> from 2018 states Netflix uses MAXPHYS=1MiB, and that in vanilla FreeBSD each queue has 128 outstanding items.

<https://lists.freebsd.org/pipermail/freebsd-arch/2020-November/020144.html> from 2020 states "Netflix has run MAXPHYS of 8MB for years"

FreeBSD 13 bumped MAXPHYS to 1MiB anyway.

https://netflixtechblog.com/serving-100-gbps-from-an-open-connect-appliance-cdb51dda3b99 states Netflix introduced a new type of mbuf that bundles 24x 4KiB pages = 96KiB, but this serves 1MiB "HTTP range request"s.

<https://papers.freebsd.org/2021/eurobsdcon/gallatin-netflix-freebsd-400gbps/>
- 400Gb/s
    - AMD EPYC 7502P (“Rome”) 32 cores
    - 2x Mellanox ConnectX-6 Dx
        - Gen4 x16, 2 full speed 100GbE ports per NIC
            - 4 x 100GbE in total
    - 18x WD SN720 NVME
    - Can't assume a given NVMe drive is dedicated to a given core.
    - 
<https://papers.freebsd.org/2022/eurobsdcon/gallatin-the_other_freebsd_optimizations-netflix/>
- 800Gb/s
    - 2x AMD EPYC 7713 64c / 128t (128c / 256t total)
    - 4x Mellanox ConnectX-6 Dx (8x 100GbE ports) NICs
    - 16x Intel Gen4 x4 14TB NVME

TODO SHOULD WATCH THOSE PRESENTATIONS NOT JUST READ SLIDES

NO INFO ON NETFLIX QUEUE SIZES, guessing 128?
Worth noting that with multiple NVMe devices you'd get one per device per core!

In [3]:
UNK_QUEUE_LEN=128

def netflix_freebsd_400Gbps_nvme_config(key_amount: KeyAmountCalcScheme) -> SimConfig:
    N_CORES=32
    N_NICS=2
    N_NIC_PORTS=N_NICS * 2
    N_NVMES=18
    return SimConfig(
        elem_size_b= 8 * MiB,
        n_queues=N_NVMES * N_CORES, # This could be reduced, if we think are aren't doing NUMA hops from NVMe->core, but it really seems like they're still hopping
        queue_len=UNK_QUEUE_LEN,
        key_amount=key_amount,
    )

def netflix_freebsd_400Gbps_network_config(key_amount: KeyAmountCalcScheme) -> SimConfig:
    N_CORES=32
    N_NICS=2
    N_NIC_PORTS=N_NICS * 2
    N_NVMES=18
    return SimConfig(
        elem_size_b= 1 * MiB, # 1MiB?
        n_queues=N_NIC_PORTS * N_CORES, # at least
        queue_len=UNK_QUEUE_LEN,
        key_amount=key_amount,
    )

def netflix_freebsd_800Gbps_nvme_config(key_amount: KeyAmountCalcScheme) -> SimConfig:
    N_CORES=128
    N_NVMES=16
    return SimConfig(
        elem_size_b= 8 * MiB,
        n_queues=N_NVMES * N_CORES,
        queue_len=UNK_QUEUE_LEN,
        key_amount=key_amount,
    )

In [4]:
run_and_log(netflix_freebsd_400Gbps_nvme_config(ExactKeyAmountPerQueue(16)))

-------
Runtime	10000
TicksBlocked	0
MaxConsumedNotFreed	7
MaxConsumedNotFreedBPerQ	56.00 MB
NQueues                 	576
MaxConsumedNotFreedTotalB	31.50 GB
QuartConsumedNotFreed	[0.0, 1.0, 2.0, 3.0, 3.5, 4.0, 5.0, 6.0, 7.0]
-------


In [5]:
run_and_log(netflix_freebsd_400Gbps_nvme_config(ExactKeyAmountPerQueue(32)))

-------
Runtime	10000
TicksBlocked	0
MaxConsumedNotFreed	3
MaxConsumedNotFreedBPerQ	24.00 MB
NQueues                 	576
MaxConsumedNotFreedTotalB	13.50 GB
QuartConsumedNotFreed	[0.0, 0.0, 1.0, 1.0, 1.5, 2.0, 2.0, 3.0, 3.0]
-------


In [6]:
run_and_log(netflix_freebsd_400Gbps_nvme_config(KeyAmountTargetingOverheadPerQueue(10 * MiB)))

KeyAmountTargetingOverhead selecting 102 in-queue keys
-------
Runtime	10000
TicksBlocked	0
MaxConsumedNotFreed	1
MaxConsumedNotFreedBPerQ	8.00 MB
NQueues                 	576
MaxConsumedNotFreedTotalB	4.50 GB
QuartConsumedNotFreed	[0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.0, 1.0, 1.0]
-------


In [7]:
def estimate_queue_size(rate_bytes_per_second, block_size_bytes, latency_milliseconds):
    # My guestimating is based on the following logic, which is similar to TCP Bandwidth Delay Product and RWIN.
    # Consider an NVMe disk. It receives a trivial amount of data to request a transfer, then either has to transfer data to or from host memory.
    # Assume for now that it is only transferring data into host memory (the host is reading).
    # Ideally, there should be enough room in the host queue for the NVMe to fill the queue *without stopping*, ...
    # ... in the TCP case it would have to stop after transmitting RWIN packets if it didn't receive an ACK for the first packet,
    # so you need to size RWIN to have enough data to saturate the network while waiting for a packet to send to the other side, and for the other side to send an ACK back.
    # i.e. RWIN needs to have enough capacity for bandwidth * (t_send + t_ack), which is usually bandwidth * RTT
    # but in the NVMe case it just has to put stuff in *another queue* and ring the doorbell.
    # print((rate_bytes_per_second * latency_milliseconds) / 1000.0)
    return ((rate_bytes_per_second * latency_milliseconds) / 1000.0) // block_size_bytes

In [8]:
# Experiment: PCIe4 NVMes claim 5000 MB/s, paging at 4KiB, and microsecond-ish latencies
one_microsecond_in_milliseconds = 1.0 / 1000.0
nvme_read_queue_size = estimate_queue_size(5000 * 1000 * 1000, 4 * KiB, one_microsecond_in_milliseconds)
print(nvme_read_queue_size)

1.0


In [9]:
nvme_of_tcp_read_queue_size = estimate_queue_size(800 * 1000 * 1000 * 1000 / 8, 1 * MiB, 30.0 / 1000.0)
print(nvme_of_tcp_read_queue_size)

2.0


In [10]:
run_and_log(SimConfig(
    elem_size_b=1 * MiB,
    n_queues=8, # one NVMe * 8 cores
    queue_len=64,
    key_amount=ExactKeyAmountPerQueue(15),
))

-------
Runtime	10000
TicksBlocked	0
MaxConsumedNotFreed	4
MaxConsumedNotFreedBPerQ	4.00 MB
NQueues                 	8
MaxConsumedNotFreedTotalB	32.00 MB
QuartConsumedNotFreed	[0.0, 0.8, 1.0, 1.6, 2.0, 2.4, 3.0, 3.2, 4.0]
-------


# High-perfomrance networking

https://www.nvidia.com/content/dam/en-zz/Solutions/networking/ethernet-adapters/connectX-6-dx-datasheet.pdf
- 200Gb/s = 25GB/s

NVIDIA recommends jumbo 9000B ethernet packets
- https://docs.nvidia.com/grace-perf-tuning-guide/optimizing-io.html
- but if we're calculating worst case longest queue length, it's actually better to assume the packets are smaller

Windows driver caps out at 4096 for each of RX/TX queue
- https://docs.nvidia.com/networking/display/winof2v237/configuring+the+driver+registry+keys

Latency = ??

Rolf paper figure 5 approximates it at 1-1.6us

But here the latency needs to actually be the latency of the link, too...

In [11]:
estimate_queue_size(25 * 1000 * 1000 * 1000, 1500, one_microsecond_in_milliseconds * 2)

33.0

# Consumer networking

https://www.intel.com/content/www/us/en/products/docs/wireless/2-4-vs-5ghz.html

Worst case: saturated ethernet or 5GHz wifi, around a gigabit = 128MBytes/s.

45Mbit required for GeForce Now:  https://www.nvidia.com/en-gb/geforce-now/system-reqs/#windows-pc
100ms worst case ping???? If you're sending 128MBytes/s to japan???

nicholsControllingQueueDelay2012 describes CoDel, which appears to have been accepted as *the* dynamic TCP sizing algorithm for buffer bloat?

Assume no jumbo packets, AFAIK they require a lot of network config to work without breaking everything?

In [12]:
estimate_queue_size(128 * 1000 * 1000, 1500, 100)

8533.0

# Key Lifetime Justification

## haasWhatModernNVMe2023

2023 paper showing having ~3000 requests outstanding on 8 NVMe SSDs saturates them in Figure 4, hence we can likely assume 3000/8 = 375 saturates one 2023 NVMe SSD.
 
- Max Queue Length = ~512 for max performance
- Latency = 60-100us, see Fig 3c
- Throughput has multiple sources: around 1million per SSD maximum stated in introduction, in practice closer to (3.75M IOPS/8 SSDs = 470k) for LeanStore in fig 1. Slower => longer key lifetime

So, consider a case where a NVMe drive has been configured for high-throughput (512 queue length) but is being used slowly in a way to maximize key lifetime.
There are only two keys, the bare minimum required which enforces each key is one whole queue length, and each transaction arrives just before the previous one is finished.
That implies ops-per-second = 1/latency.

In [18]:
def max_key_lifetime(*, elems_per_key, initial_latency_s, elems_per_s):
    return initial_latency_s + elems_per_key / elems_per_s

def print_ms(seconds):
    print(f"{seconds*1000:.2f}ms")

NVME_MAX_QUEUE_LENGTH = 500
NVME_LATENCY = 100 * us
print_ms(max_key_lifetime(
    elems_per_key = NVME_MAX_QUEUE_LENGTH,
    initial_latency_s = NVME_LATENCY,
    elems_per_s = 1 / NVME_LATENCY,
))

50.10ms


## patelSplitwiseEfficientGenerative2024

Table IV shows time-between-tokens are ~50ms, BUT time to first token is up to 200ms. In practice this sort of thing should be uploaded to the GPU anyway...
Even local models https://arxiv.org/pdf/2410.08391v1 (WITHDRAWN) take waay longer (multiple seconds) to generate tokens. Prudent to use the pool model here.

## Hypothetical task-based architectures

### GPUs

Consider a GPU rendering a real-time interactive video game at 30 frames per second (33ms). In these cases the CPU and GPU typically run in parallel, with the CPU preparing data for one frame as the GPU is rendering the current frame. If the CPU exposed each frame's data all at once as a single task, (which would imply it's all CPU-resident persistent mappings, which is unlikely) the memory would be exposed for ~33ms. This also assumes the GPU driver exposes that memory at a time very close to frame computation start, such as command buffer build time.


### High-res video decode

In a similar fashion high resolution video decoding may have dedicated accelerators, which would have 42ms to decode each frame of video at the standard 24fps cinematic frame rate.